# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart workflow
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Setup
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))

from gu_toolkit import *


## 1) Quickstart: Minimal interactive figure

This pattern covers most exploratory notebook work:
- create a figure,
- define one or more plots,
- bind parameters by name,
- render.


In [ ]:
x, a, b = sp.symbols("x a b")
fig = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=500)

with fig:
    plot(x, a*sp.sin(b*x), parameters=[a, b], id="wave", label="a*sin(bx)")
    parameter(a, value=1.0, min=-2.0, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.2, max=4.0, step=0.1)
    set_title("Quickstart: Interactive sine")

fig


## 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
# Compact utility snippets
with fig:
    print("x-range:", get_x_range())
    print("y-range:", get_y_range())
    print("param snapshot:", params.snapshot())


## 3) Parameter patterns you will reuse often

### A) Explicit slider bounds and defaults
Useful for reproducible demos.


In [ ]:
x, amp, freq, phase = symbols("x amp freq phase")
fig2 = Figure(x_range=(-10, 10), y_range=(-2, 2), sampling_points=600)

with fig2:
    plot(x, amp*sp.cos(freq*x + phase), parameters=[amp, freq, phase], id="cosine")
    parameter(amp, value=1.0, min=0.0, max=2.0, step=0.05)
    parameter(freq, value=1.0, min=0.1, max=3.0, step=0.05)
    parameter(phase, value=0.0, min=-pi, max=pi, step=0.1)
    set_title("Cosine with amplitude/frequency/phase controls")
fig2

### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
expr.free_symbols

In [ ]:
f = numpify(expr, vars=['x', 'A', 'k', 'phi'])
f

In [ ]:
fig3 = Figure(x_range=(-8, 8), y_range=(-3, 3), sampling_points=700)
with fig3:
    plot(x, A*sin(k*x + phi), parameters=[A, k, phi], id="symbolic-wave")
    parameter(A, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(k, value=1.0, min=0.2, max=4.0, step=0.1)
    parameter(phi, value=0.0, min=-pi, max=pi, step=0.1)
    set_title("Symbolic expression with dynamic parameters")
fig3


## 4) Debugging and introspection helpers

These checks are useful when a notebook cell is not behaving as expected.


In [ ]:
with fig3:
    print("Current params:", params.snapshot())
print("Figure has", len(fig3.plots), "plot(s)")
print("Plot ids:", list(fig3.plots.keys()))

plot_obj = fig3.plots["symbolic-wave"]
print("Plot label:", plot_obj.label)
print("Has cached samples:", plot_obj.cached_samples is not None)

#BUG 
#---------------------------------------------------------------------------
#AttributeError                            Traceback (most recent call last)
#Cell In[21], line 8
#      6 plot_obj = fig3.plots["symbolic-wave"]
#      7 print("Plot label:", plot_obj.label)
#----> 8 print("Has cached samples:", plot_obj.cached_samples is not None)
#
#AttributeError: 'Plot' object has no attribute 'cached_samples'

## 5) Cool applications

### Application 1: Harmonic synthesizer
A compact additive synthesis visualization.


In [ ]:
fig4 = Figure(x_range=(-2*pi, 2*pi), y_range=(-4, 4), sampling_points=900)
with fig4:
    plot(x, a[1]*sin(x) + a[2]*sin(2*x) + a[3]*sin(3*x), id="harmonic")
    parameter(a[1], value=1.0, min=-2.0, max=2.0, step=0.1)
    parameter(a[2], value=0.0, min=-2.0, max=2.0, step=0.1)
    parameter(a[3], value=0.0, min=-2.0, max=2.0, step=0.1)
    set_title("Harmonic synthesizer")
fig4

### Application 2: Gaussian mixture explorer
Great for intuition about peak width, center, and mixture weights.


In [ ]:
w=SymbolFamily('w')
s=SymbolFamily('s')
m=SymbolFamily('m')

expr = w[1]*exp(-sp.Rational(1, 2)*((x - m[1])/s[1])**2) + w[2]*exp(-sp.Rational(1, 2)*((x - m[2])/s[2])**2)

fig5 = Figure(x_range=(-8, 8), y_range=(0, 2), sampling_points=800)
with fig5:
    plot(x, expr, id="mixture")
    parameter(w[1], value=1.0, min=0.0, max=1.5, step=0.05)
    parameter(w[2], value=0.8, min=0.0, max=1.5, step=0.05)
    parameter(m[1], value=-1.5, min=-5.0, max=5.0, step=0.1)
    parameter(m[2], value=1.5, min=-5.0, max=5.0, step=0.1)
    parameter(s[1], value=1.0, min=0.2, max=3.0, step=0.1)
    parameter(s[2], value=0.8, min=0.2, max=3.0, step=0.1)
    set_title("Gaussian mixture explorer")
fig5

## 6) Practical checklist

When prototyping in notebooks:
- Start with one plot and one parameter.
- Confirm `params.snapshot()` updates while sliders move.
- Increase `sampling_points` only when needed.
- Use clear `id=` values for plots you update repeatedly.

This notebook is intended as a durable daily-use reference.


## 7) Project-019: Tabbed multi-view workspace (phase 5/6 showcase)

This section demonstrates:
- creating multiple named views/tabs via `add_view(...)`,
- targeting plots and info cards to specific views,
- temporary view scoping with `with fig.view("...")`,
- snapshot/codegen support for multi-view state.


In [ ]:
x, a, b = sp.symbols("x a b")
fig_multi = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_multi.title = "Project-019 multi-view demo"

# Add an alternate view with independent default ranges/labels
fig_multi.add_view("frequency", x_range=(-12, 12), y_range=(-2, 2), x_label="ω", y_label="magnitude")

with fig_multi:
    # Shared parameters drive both views
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    # Plot in both views
    plot(x, a*sp.sin(x) + b, parameters=[a, b], id="shared-wave", view=("main", "frequency"))

    # View-scoped plot via context manager
    with fig_multi.view("frequency"):
        plot(x, sp.cos(2*x), id="freq-only", color="purple")

    # Shared and view-scoped info cards
    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Frequency note:</b> only visible on 'frequency'", id="freq-card", view="frequency")

fig_multi

In [ ]:
print("Active view:", fig_multi.active_view_id)
print("Views:", tuple(fig_multi.views.keys()))
print("shared-wave memberships:", fig_multi.plots["shared-wave"].views)

snap = fig_multi.snapshot()
print("snapshot.active_view_id:", snap.active_view_id)
print("snapshot view ids:", tuple(v.id for v in snap.views))
print("snapshot scoped cards:", [(c.id, c.view_id) for c in snap.info_cards])

code = fig_multi.to_code()
print("code contains add_view:", "fig.add_view('frequency'" in code)
print("code contains scoped info:", "view='frequency'" in code)